In [1]:
from multiprocessing import Pool
import numpy as np
import random
from itertools import combinations as cb
import math
from copy import deepcopy as dc
from tqdm import tqdm

In [2]:
"""Evaluate Function """
class Evaluate:
    def __init__(self):
        None
    def evaluate(self,gen):
        None
    def check_dimentions(self,dim):
        None

"""Common Function"""
def random_search(n,dim):
    """
    create genes list
    input:{ n: Number of population, default=20
            dim: Number of dimension
    }
    output:{genes_list → [[0,0,0,1,1,0,1,...]...n]
    }
    """
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens

In [3]:
"""BGA"""
def suddn(li,n_li,num):#突然変異
    l1= [random.choice(n_li) for i in range(num)]
    l2= [random.choice([0,1]) for i in range(num)]
    al_li=dc(li)
    for i in range(len(l1)):
        al_li[l1[i]]=l2[i]
    #li=''.join(_d)
    return al_li

def BGA(Eval_Func,n=20,m_i=300,mutation=0.05,minf=0,dim=None,prog=False,mp=None):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            m_i: Number of max iteration, default=300
            mutation: Probability of mutation, default=0.05(5%)
            minf: minimazation flag, default=0, 0=maximization, 1=minimazation
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            }
    output:{Best value: type float 0.967
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    estimate=Eval_Func().evaluate
    if dim==None:
        dim=Eval_Func().check_dimentions(dim)
    gens=random_search(n,dim)
    fit=[0 for i in range(n)]
    num_li=range(dim)
    #flag=dr
    best_val=float("-inf") if minf == 0 else float("inf")#minf==0のときは最大化なので-infを初期ベストにし、全部0の部分集合を初期ベストにする
    best_pos=[0]*dim
    gens_dict={tuple([0]*dim):float("-inf") if minf == 0 else float("inf")}
    prop=mutation

    if prog:
        miter=tqdm(range(m_i))
    else:
        miter=range(m_i)

    for it in miter:
        if mp !=None:
            fit=[gens_dict[tuple(g)]  if tuple(g) in gens_dict  else(float("-inf") if minf == 0 else float("inf")) for g in gens]
            alter_gens=[k for k,g in enumerate(gens) if tuple(g) not in gens_dict]
            #print(len(alter_gens))
            with Pool(mp) as p:
                alter_fit = p.map(estimate,[gens[k] for k in alter_gens])
            print(alter_fit)
            z=0
            for zz in range(len(fit)):
                if zz in alter_gens:
                    fit[zz]=alter_fit[z]
                    gens_dict[tuple(gens[zz])]=alter_fit[z]
                    z+=1
                else:pass
                if best_val < fit[zz] if minf==0 else best_val > fit[zz]:
                    best_val=dc(fit[zz])
                    best_pos=dc(gens[zz])
        else:
            for i,gen in enumerate(gens):
                ################################

                    ################################
                    if tuple(gen) in gens_dict:
                        v=gens_dict[tuple(gen)]
                    else:
                        score=estimate(gen)
                        gens_dict[tuple(gen)]=score
                    fit[i]=score
                    if best_val < score if minf==0 else best_val > score:
                        best_val=dc(score)
                        best_pos=dc(gen)
        alter_gens=sorted(gens,reverse=True)[:2]
        t1=random.randint(1,len(gens[0])-2)
        t2=random.randint(t1,len(gens[0])-1)

        fit_ind=np.argsort(fit)[::-1][:n//2]
        sample_num=random.sample(list(cb(fit_ind,2)),n-2)
        qgens=[suddn(gens[s][:t1]+gens[m][t1:t2]+gens[s][t2:],num_li,dim//3) if np.random.choice([0,1],size=1,p=[1-prop,prop])[0]==1
               else gens[s][:t1]+gens[m][t1:t2]+gens[s][t2:] for s,m in sample_num]
        gens=[]
        gens.extend(qgens)
        gens.append(alter_gens[0])
        gens.append(alter_gens[1])
    print(len(gens_dict))
    return best_val,best_pos,best_pos.count(1)

In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
 
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import numpy as np

from sklearn import svm
 
SEED = 2018
np.random.seed(SEED)
df_train = pd.read_csv("CSS10_Spec128.csv")
X = np.array(df_train.drop(['label'],axis=1))
y = np.array(df_train['label'])
from sklearn.model_selection import train_test_split
df=df_train
(a,b)=np.shape(df)
print(a,b)
data = df.values[:,0:b-1]
label = df.values[:,b-1]
# trainX=data
# trainy=label
cross=4
test_size=(1/cross)
trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size)

7000 129


In [5]:
if __name__ == '__main__':
    from sklearn import svm
    from time import time

    np.random.seed(20)
    tr_d=trainX
    te_d=testX
    tr_l=trainy
    te_l=testy

In [6]:
class Evaluate:#setting class
        def __init__(self):#set train_data,label,test_data,label
            self.train_l=tr_l
            self.train_d=tr_d
            self.test_l=te_l
            self.test_d=te_d
        def evaluate(self,gen):
            """
            Setting of evaluation function.
            Here, the correct answer rate is used.
              anser_label/all_label
            """
            mask=np.array(gen) > 0
            al_data=np.array([al[mask] for al in self.train_d])
            al_test_data=np.array([al[mask] for al in self.test_d])
            #↑masking with [01]sequence list
            res=RandomForestClassifier(n_jobs = -1, verbose = 0, n_estimators=5, criterion='entropy', random_state = 0).fit(al_data,self.train_l).predict(al_test_data)
            return np.count_nonzero(self.test_l==res)/len(self.test_l)
            #↑evaluate with fittness function
        def check_dimentions(self,dim):#check number of all feature
            if dim==None:
                return len(self.train_d[0])
            else:
                return dim

In [17]:
%%time
s,g,l=BGA(Eval_Func=Evaluate, n=2, m_i=1)

3
CPU times: user 421 ms, sys: 17.7 ms, total: 438 ms
Wall time: 655 ms


In [18]:
l

48

In [19]:
List = []

for i in range(0,len(g)):
    if g[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("CSS10_Spec128.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,label
0,0.000725,0.000390,0.001063,0.045006,0.088202,0.041129,0.018479,0.050937,0.081694,0.065240,...,0.066671,0.041071,0.019739,0.037832,0.034691,0.037083,0.040290,0.035810,0.033180,Chinese
1,0.005059,0.002901,0.005581,0.217670,0.280215,0.189792,0.146428,0.095449,0.252080,0.238911,...,0.066760,0.081203,0.067477,0.050477,0.057735,0.095075,0.205824,0.407305,0.089081,Chinese
2,0.050511,0.033339,0.020180,0.193343,0.242709,0.228159,0.061330,0.097945,0.509217,0.510332,...,0.064809,0.037791,0.044022,0.045836,0.042054,0.081357,0.026522,0.028574,0.012089,Chinese
3,0.001279,0.000901,0.003375,0.200739,0.168802,0.077728,0.077944,0.106558,0.437945,0.378038,...,0.064433,0.095225,0.025008,0.055018,0.073949,0.109730,0.023716,0.014288,0.014407,Chinese
4,0.001710,0.001331,0.002648,0.163129,0.289433,0.295286,0.051800,0.170620,0.555679,0.239185,...,0.238380,0.123092,0.066073,0.268126,0.144939,0.131616,0.209389,0.256350,0.106921,Chinese
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.057926,2.879856,20.679771,0.379650,1.958890,5.435989,7.585841,2.496896,6.943397,4.972301,...,0.027533,0.030768,0.010501,0.007672,0.004163,0.001669,0.008678,0.006215,0.011008,Spanish
6996,0.078999,4.854642,16.547379,0.576475,3.127383,4.378862,5.144395,4.036949,4.024424,5.693283,...,0.024001,0.018118,0.005046,0.004128,0.002328,0.000990,0.007153,0.005911,0.008319,Spanish
6997,0.236640,3.645541,11.302727,0.851596,2.084428,3.044496,5.026220,4.140211,2.992726,3.786492,...,0.028762,0.025592,0.007572,0.005612,0.002758,0.001001,0.017653,0.019413,0.004149,Spanish
6998,0.118839,2.011996,15.797884,0.504213,1.365581,5.007298,9.100726,4.591431,7.282707,3.693865,...,0.037475,0.047870,0.006598,0.006576,0.003559,0.000874,0.004784,0.004022,0.010245,Spanish


In [20]:
df_train1.to_csv('BGA128.csv')